<a href="https://colab.research.google.com/github/bermudezfc/CCMACLRL_EXAM/blob/main/ML_Midterm_Exam_Bermudez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [635]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor

In [636]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv(dataset_url)

In [637]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [638]:
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [639]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [640]:
sf

,id,price
0,188533,43878.016
1,188534,43878.016
2,188535,43878.016
3,188536,43878.016
4,188537,43878.016
...,...,...
125685,314218,43878.016
125686,314219,43878.016
125687,314220,43878.016
125688,314221,43878.016


In [641]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [642]:
df['fuel_type'].value_counts()

,count
fuel_type,
Gasoline,165940
Hybrid,6832
E85 Flex Fuel,5406
Diesel,3955
–,781
Plug-In Hybrid,521
not supported,15


In [643]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
20278,20278,Mercedes-Benz,E-Class E55 AMG,2005,232492,Gasoline,469.0HP 5.4L 8 Cylinder Engine Gasoline Fuel,5-Speed A/T,Gray,Black,At least 1 accident or damage reported,Yes,46900
40307,40307,Dodge,Ram 1500 SLT Mega Cab,2007,64500,Gasoline,345.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,Blue,Gray,At least 1 accident or damage reported,Yes,16000
125570,125570,Porsche,911 Carrera,2011,17000,Gasoline,430.0HP 3.8L Flat 6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Green,Black,None reported,Yes,94995
117248,117248,Honda,CR-V EX,2008,186040,Gasoline,160.0HP 2.4L 4 Cylinder Engine Gasoline Fuel,M/T,Gray,Black,At least 1 accident or damage reported,Yes,6900
86616,86616,Ford,F-150 Raptor,2021,32957,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,Automatic,Oxford White,Rhapsody Blue,None reported,NaN,49883
80350,80350,Porsche,Macan S,2018,71273,Gasoline,340.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Silver,Black,None reported,Yes,23500
151266,151266,Buick,Regal Turbo - Premium 1,2007,161900,Gasoline,220.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Beige,Beige,At least 1 accident or damage reported,Yes,7500
99287,99287,Lexus,IS 250C Base,2010,102665,Gasoline,204.0HP 2.5L V6 Cylinder Engine Gasoline Fuel,6-Speed M/T,Black,Beige,None reported,Yes,11499
188275,188275,Jeep,Gladiator Overland,2020,43000,Gasoline,285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,34000
134949,134949,Audi,A4 2.0T Premium,2013,66150,Gasoline,211.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,Beige,None reported,Yes,16000


In [644]:
print(len(df))

188533


In [645]:
df['clean_title'].value_counts()

,count
clean_title,
Yes,167114


In [646]:
df['clean_title'] = df['clean_title'].fillna('No')
dt['clean_title'] = df['clean_title'].fillna('No')

In [647]:
df['clean_title'].value_counts()

,count
clean_title,
Yes,167114
No,21419


In [648]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
93664,93664,Nissan,350Z Enthusiast,2009,149000,Gasoline,300.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,5-Speed A/T,Black,Black,None reported,Yes,8500
178791,178791,Chevrolet,Camaro 2SS,2017,51850,Gasoline,455.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,None reported,Yes,44700
11461,11461,Mercedes-Benz,E-Class E 350 4MATIC,2008,160000,Gasoline,268.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,–,None reported,Yes,5995
166782,166782,Land,Rover Range Rover Sport HST MHEV,2020,19000,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Silver,Black,None reported,Yes,66000
127259,127259,BMW,335 i,2007,95450,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,17000
124483,124483,Ford,Escape XLT,1993,170000,Gasoline,–,A/T,White,Gray,At least 1 accident or damage reported,Yes,9499
62065,62065,Cadillac,Escalade ESV Platinum,2016,119000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,Black,None reported,Yes,32999
65135,65135,Ford,F-150 XLT,2022,23842,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,Iconic Silver Metallic,Black,None reported,Yes,48149
167156,167156,Chevrolet,Camaro 1SS,2019,16700,Gasoline,460.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Gray,Black,None reported,Yes,45000
54222,54222,Mercedes-Benz,C-Class C 300 4MATIC Luxury,2013,93500,Gasoline,248.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes,20490


In [649]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [650]:
df['accident'].value_counts()

,count
accident,
None reported,144514
At least 1 accident or damage reported,41567


In [651]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [652]:
print(len(df))

188533


In [653]:
# Car Age

from datetime import datetime

current_year = datetime.now().year
df['car age'] = current_year - df['model_year']
dt['car age'] = current_year - df['model_year']


In [654]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car age
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3


In [655]:
dt.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,car age
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes,17
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes,22
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,Yes,22
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,Yes,7
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,3


In [656]:
print(len(df))

188533


In [657]:
print(len(dt))

125690


In [658]:
dt.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3383
engine,0
transmission,0
ext_col,0
int_col,0


In [659]:
# For df dataframe
df['liter'] = pd.to_numeric(df['engine'].str.extract(r'(?P<liter>\d+\.?\d*)\s*(?:L|Liter)')['liter'], errors='coerce')

# For dt dataframe
dt['liter'] = pd.to_numeric(dt['engine'].str.extract(r'(?P<liter>\d+\.?\d*)\s*(?:L|Liter)')['liter'], errors='coerce')


In [660]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

label_encoder = LabelEncoder()
df['brand'] = label_encoder.fit_transform(df['brand'])
dt['brand'] = label_encoder.fit_transform(dt['brand'])

In [661]:
label_encoder = LabelEncoder()
df['model'] = label_encoder.fit_transform(df['model'])
dt['model'] = label_encoder.fit_transform(dt['model'])


In [662]:
label_encoder = LabelEncoder()
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])
dt['fuel_type'] = label_encoder.fit_transform(dt['fuel_type'])

In [663]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car age,liter
0,0,31,495,2007,213000,2,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17,1.6
1,1,28,930,2002,143250,2,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22,3.9
2,2,9,1575,2002,136731,1,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22,5.3
3,3,16,758,2017,19500,2,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7,5.0
4,4,36,1077,2021,7388,2,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3,2.0


In [664]:
df['accident'].value_counts()

,count
accident,
None reported,144514
At least 1 accident or damage reported,41567


In [665]:

df['accident'] = label_encoder.fit_transform(df['accident'])

dt['accident'] = label_encoder.fit_transform(dt['accident'])

In [666]:
df['clean_title'] = label_encoder.fit_transform(df['clean_title'])
dt['clean_title'] = label_encoder.fit_transform(dt['clean_title'])

In [667]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [668]:
label_encoder = LabelEncoder()
df['transmission'] = label_encoder.fit_transform(df['transmission'])
dt['transmission'] = label_encoder.fit_transform(dt['transmission'])

In [669]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car age,liter
0,0,31,495,2007,213000,2,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,38,Yellow,Gray,1,1,4200,17,1.6
1,1,28,930,2002,143250,2,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,38,Silver,Beige,0,1,4999,22,3.9
2,2,9,1575,2002,136731,1,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,38,Blue,Gray,1,1,13900,22,5.3
3,3,16,758,2017,19500,2,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,49,Black,Black,1,1,45000,7,5.0
4,4,36,1077,2021,7388,2,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,23,Black,Beige,1,1,97500,3,2.0


In [670]:
dt.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,car age,liter
0,188533,26,1388,2015,98000,2,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,16,White,Beige,1,1,17,2.0
1,188534,26,1375,2020,9142,3,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,31,Silver,Black,1,1,22,3.0
2,188535,14,636,2022,28121,2,3.5L V6 24V PDI DOHC Twin Turbo,3,White,Ebony,1,1,22,3.5
3,188536,3,182,2016,61258,2,2.0 Liter TFSI,39,Silician Yellow,Black,1,1,7,2.0
4,188537,3,181,2018,59000,2,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,38,Gray,Black,1,1,3,2.0


In [671]:
mean_imputer = SimpleImputer(strategy='mean')
df['liter'] = mean_imputer.fit_transform(df[['liter']])
dt['liter'] = mean_imputer.fit_transform(dt[['liter']])

In [672]:
df.drop(columns = ['id', 'engine', 'ext_col', 'int_col'], inplace=True)
dt.drop(columns = ['id', 'engine', 'ext_col', 'int_col'], inplace=True)

In [673]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,transmission,accident,clean_title,price,car age,liter
132656,39,265,2021,18500,2,23,1,1,59980,3,5.6
63611,26,1435,2016,57000,2,49,1,1,70000,8,5.0
61707,14,677,2022,2894,2,2,1,1,101000,2,7.3
185499,4,44,2017,83000,2,38,0,1,18000,7,2.0
155002,39,785,2022,7900,2,38,1,1,89000,2,3.8


In [674]:
dt.sample(5)

,brand,model,model_year,milage,fuel_type,transmission,accident,clean_title,car age,liter
42276,52,867,2020,75600,2,32,0,1,17,2.0
7866,14,644,2020,93400,2,16,1,1,4,3.5
37064,52,412,2006,145000,2,38,0,0,4,3.7
89842,45,529,2021,18160,2,32,1,0,6,6.7
23444,52,864,2019,9500,2,38,1,1,7,3.5


In [675]:
from sklearn.model_selection import train_test_split

y = df["price"]



x = df.drop(['price'], axis = 1)



X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2, random_state=42)

In [676]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [677]:
y_pred_train = linear_reg.predict(X_train)
y_pred_test = linear_reg.predict(X_test)

In [678]:
train_mse = mean_squared_error(y_train, y_pred_train, squared=False)
test_mse = mean_squared_error(y_test, y_pred_test, squared=False)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Training MSE: {train_mse}")
print(f"Testing MSE: {test_mse}")
print(f"Training R2 Score: {train_r2}")
print(f"Testing R2 Score: {test_r2}")

Training MSE: 76109.32272585986
Testing MSE: 70588.53965097434
Training R2 Score: 0.09139828499385938
Testing R2 Score: 0.10400643520268893


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [679]:
id = sf.pop('id')
y_pred = linear_reg.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'price': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('results.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
